Import Statements

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import data.constants as const


In [ ]:
FILE_PATH = f'{const.SAVE_DIR}/worst_failure.csv'
#FILE_PATH = f'{const.SAVE_DIR}/saveData1.csv'

df = pd.read_csv(FILE_PATH)

In [ ]:
# Net Worth & returns
plt.figure(figsize=(14,8), dpi=120) 

roll_df = df[['Stock Returns', 'Bond Returns','RE Returns']].rolling(window=6).mean()

ax1 = plt.gca() # get the axis
ax2 = ax1.twinx() # create another axis that shares the same x-axis

ax1.set_ylim(bottom=-df['Net Worth'].max(), top=df['Net Worth'].max())
min_return, max_return = min(roll_df['Stock Returns'].min(),roll_df['RE Returns'].min()), max(roll_df['Stock Returns'].max(),roll_df['RE Returns'].max())
ax2.set_ylim(bottom=min_return, top=max_return+(max_return-min_return))

ax1.plot(df['Date'],df["Net Worth"], color='skyblue')
ax2.plot(df['Date'],roll_df["Stock Returns"], color='darkorange',label='Stock')
ax2.axhline(const.EQUITY_MEAN ** (1/4)-1, color='darkorange', lw=1)
ax2.plot(df['Date'],roll_df["Bond Returns"], color='forestgreen',label='Bond')
ax2.axhline(const.BOND_MEAN ** (1/4)-1, color='forestgreen', lw=1)
ax2.plot(df['Date'],roll_df["RE Returns"], color='indianred',label='Real Estate')
ax2.axhline(const.RE_MEAN ** (1/4)-1, color='indianred', lw=1)
plt.legend() 

ax1.set_xlabel('Year')
ax1.set_ylabel('Net Worth', color='skyblue')
ax2.set_ylabel('Returns')

Check rates

In [ ]:
# Inflation
years = math.trunc(df['Date'].max()-df['Date'].min())
product = list(df['Inflation'])[-1]
annualized = pow(product, 1 / years)

plt.plot(df['Date'],df["Inflation"],label=f'Generated: {annualized:.3f}')

def expected_ls(val):
    qt_val = val ** (1/4)
    ls = [qt_val]
    for _ in df["Inflation"]:
        ls.append(ls[-1] * qt_val)
    ls.pop()
    return ls

expected_inflation_ls = expected_ls(const.INFLATION_MEAN)
low_ls = expected_ls(const.INFLATION_ANNUAL_LOW)
high_ls = expected_ls(const.INFLATION_ANNUAL_HIGH)

plt.plot(df['Date'],expected_inflation_ls,label=f'Expected: {const.INFLATION_MEAN}',color='darkorange')
plt.plot(df['Date'],low_ls,color='bisque',label=f'Bounds: {const.INFLATION_ANNUAL_LOW} to {const.INFLATION_ANNUAL_HIGH}')
plt.plot(df['Date'],high_ls,color='bisque')
plt.yscale('log')
plt.legend() 


In [ ]:
years = math.trunc(df['Date'].max()-df['Date'].min())
yr_sets_of_qt_returns=np.array_split((df['Stock Returns']+1),years)
yr_returns = []
for set in yr_sets_of_qt_returns:
        yr_returns.append(np.product(set))
plt.hist(yr_returns,15,label='4 Quart Sets')
print(np.mean(yr_returns))
plt.legend()

data = {'Generated':[np.mean(yr_returns)],
        'Expected':[const.EQUITY_MEAN]}

pd.DataFrame(data, index=['avg'
                               ])

In [ ]:
# Allocations over time

ax1 = plt.gca() # get the axis
ax2 = ax1.twinx() # create another axis that shares the same x-axis

ax1.set_ylim(bottom=-df['Net Worth'].max(), top=df['Net Worth'].max())
ax2.set_ylim(bottom=0, top=1.5)

ax1.plot(df['Date'],df["Net Worth"])
#ax1.axhline(y=param_vals['Equity Target'], color='r', linestyle='-') keeps drawing a line at 0, not sure why
ax2.plot(df['Date'],df["Stock Alloc"],label='Stock Alloc')
ax2.plot(df['Date'],df["Bond Alloc"],label='Bond Alloc')
ax2.plot(df['Date'],df["RE Alloc"],label='RE Alloc')
plt.legend()
